In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 12.7 GB  | Proc size: 158.8 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total 16280MB


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
import numpy as np
from keras.layers import LSTM,Dropout,Dense,Activation
from keras.models import Sequential,Model,load_model
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from music21 import converter, instrument, note, chord,stream
import glob

Using TensorFlow backend.


In [ ]:
def import_data():
  notes = list()
  for file in glob.glob('/gdrive/My Drive/midi_songs/*.mid'):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if(parts):
      notes_to_parse = parts.parts[0].recurse()
    else:
      notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
      if(isinstance(element,note.Note)):
        notes.append(str(element.pitch))
      elif(isinstance(element,chord.Chord)):
        notes.append('.'.join(str(n) for n in element.normalOrder))
  return notes

In [ ]:
notes = import_data()
n_vocab = len(set(notes))

In [ ]:
def generate_input_for_network(notes,n_vocab):
  sequence_length = 100
  pitchnames = sorted(set(item for item in notes)) #Umquie vales in the training set
  # create a dictionary to map pitches to integers
  notes_to_int = dict((pitchnames[i], i) for i in range(len(pitchnames)))
  network_in_X = list()
  network_out_y = list()
  for i in range(0,len(notes)-sequence_length):
    sequence_in = notes[i:i+sequence_length]
    sequence_out = notes[i+sequence_length]
    network_in_X.append([notes_to_int[char] for char in sequence_in])
    network_out_y.append(notes_to_int[sequence_out])
  n_patterns = len(network_in_X)
  network_in_X = np.reshape(network_in_X,(n_patterns,sequence_length,1))
  network_in_X_norm = network_in_X / float(n_vocab)
  network_out_y = to_categorical(network_out_y)
  return network_in_X,network_in_X_norm,pitchnames

In [ ]:
network_in_X,network_in_X_norm,pitchnames = generate_input_for_network(notes,n_vocab)


In [ ]:
def load_lstm_model(network_in_X,n_vocab):
  model = Sequential()
  model.add(LSTM(256,input_shape=(network_in_X.shape[1],network_in_X.shape[2]),return_sequences=True))
  model.add(Dropout(0.3))
  model.add(LSTM(512,return_sequences=True))
  model.add(Dropout(0.3))
  model.add(LSTM(256))
  model.add(Dense(256))
  model.add(Dropout(0.3))
  model.add(Dense(n_vocab))
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy',optimizer='rmsprop')
  model.load_weights('/gdrive/My Drive/Intermediate_weights_music_gen_v4/weights-improvement-99-0.6228-bigger.hdf5')
  return model

In [ ]:
model=load_lstm_model(network_in_X_norm,n_vocab)

In [ ]:
def generate_sequence(model,network_in_X,n_vocab,pitchnames):
  start = np.random.randint(0, len(network_in_X)-1)
  int_to_note = dict((i, pitchnames[i]) for i in range(len(pitchnames)))
  pattern = list(network_in_X[start])
  prediction_output = []
  # generate 500 notes
  for note_index in range(100):
      prediction_input = np.reshape(pattern, (1, len(pattern), 1))
      prediction_input = prediction_input / float(n_vocab)
      prediction = model.predict(prediction_input, verbose=0)
      index = np.argmax(prediction)
      result = int_to_note[index]
      prediction_output.append(result)
      pattern.append(index)
      pattern = pattern[1:len(pattern)]
  return prediction_output

In [ ]:
prediction_output = generate_sequence(model,network_in_X,n_vocab,pitchnames)

In [ ]:
def generate_notes(prediction_output):
  offset = 0
  output_notes = []

  # create note and chord objects based on the values generated by the model
  for pattern in prediction_output:
      # pattern is a chord
  #     print(pattern)
      if (('.' in pattern) or pattern.isdigit()):
          notes_in_chord = pattern.split('.')
          notes = []
          for current_note in notes_in_chord:
              new_note = note.Note(int(current_note))
              new_note.storedInstrument = instrument.Piano()
              notes.append(new_note)
          new_chord = chord.Chord(notes)
          new_chord.offset = offset
          output_notes.append(new_chord)
      # pattern is a note
      else:
          new_note = note.Note(pattern)
          new_note.offset = offset
          new_note.storedInstrument = instrument.Piano()
          output_notes.append(new_note)
      # increase offset each iteration so that notes do not stack
      offset += 0.5
  return output_notes

In [ ]:
output_notes = generate_notes(prediction_output)

In [ ]:
def generate_midi_file(output_notes):
  midi_stream = stream.Stream(output_notes)
  midi_stream.write('midi', fp='/gdrive/My Drive/test_output_final3.mid')

In [ ]:
generate_midi_file(output_notes)